In [25]:
# Importing the libraries
import transformers
import datasets
import numpy as np
import pandas as pd
import tensorflow as tf

In [26]:
# Importing the transformers modules
from transformers import TFAutoModel, AutoTokenizer, GenerationConfig,TFAutoModelForSequenceClassification
from datasets import load_dataset

In [27]:
# Downloading the toxicity dataset
toxicity_dataset_name = "Arsive/toxicity_classification_jigsaw"

dataset = load_dataset(toxicity_dataset_name)

In [28]:
# Downloading the tokenizer and model
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModel.from_pretrained(checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [29]:
# Testing the hugging face tokenizer
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]

inputs = tokenizer(raw_inputs,return_tensors="tf",padding=True,truncation=True)
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])>}


In [30]:
# Testing the model(General model used here by using TFAutoModel)

outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


In [31]:
# Testing the sequence model(used by TFAutoModelForSequenceClasification) - contains the head(output layer) for sequence classification

model_1 = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs_1 = model_1(inputs)
print(outputs_1.logits)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tf.Tensor(
[[-1.5616996  1.6130385]
 [ 4.169161  -3.3463957]], shape=(2, 2), dtype=float32)


In [32]:
# Convert the logits into probabilities
predictions = tf.math.softmax(outputs_1.logits,axis=-1)
print(predictions)

tf.Tensor(
[[4.0127516e-02 9.5987248e-01]
 [9.9945575e-01 5.4425059e-04]], shape=(2, 2), dtype=float32)


In [33]:
# Convert the probabilities into labels
for i in range(predictions.shape[0]):
    predicted_label = tf.argmax(predictions, axis=-1)[i]
    predicted_label_name = model.config.id2label[int(predicted_label)]
    print("Predicted label:", predicted_label_name)

Predicted label: POSITIVE
Predicted label: NEGATIVE
